# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
!pip install lightgbm

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import time
from sklearn.model_selection import GridSearchCV

## Подготовка данных

In [3]:
dataset = pd.read_csv('/datasets/autos.csv')

In [4]:
dataset.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


**Признаки**

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак**
- Price — цена (евро)

Изучите данные. Заполните пропущенные значения и обработайте аномалии в столбцах. Если среди признаков имеются неинформативные, удалите их.


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [6]:
dataset.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [7]:
dataset.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Подсчёт пропусков по полям. Пропуски есть в полях: VehicleType, Gearbox, Model, FuelType, Repaired

Пропуски имеет смысл заполнить наиболее часто встречающимися значениями полей

In [8]:
dataset["VehicleType"].value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

In [9]:
dataset["Gearbox"].value_counts()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

In [10]:
dataset["Model"].value_counts()

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
serie_1                   2
range_rover_evoque        2
Name: Model, Length: 250, dtype: int64

In [11]:
dataset["FuelType"].value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

In [12]:
dataset["Repaired"].value_counts()

no     247161
yes     36054
Name: Repaired, dtype: int64

In [13]:
dataset["VehicleType"] = dataset["VehicleType"].fillna('sedan')
dataset["Gearbox"] = dataset["Gearbox"].fillna('manual')
dataset["Model"] = dataset["Model"].fillna('golf')
dataset["FuelType"] = dataset["FuelType"].fillna('petrol')
dataset["Repaired"] = dataset["Repaired"].fillna('no')

In [14]:
dataset['Price'].describe()

count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

Описательная статистика по целевому признаку (цене)

In [15]:
dataset = dataset.query('Price > 0')

Убираю значения с нулевым прайсом

In [16]:
dataset = dataset.query('Power > 0')

Убираю записи с нулевой мощностью

In [17]:
dataset = dataset.drop(['RegistrationMonth', 'DateCrawled', 'NumberOfPictures', 'DateCreated', 'PostalCode', 'LastSeen'], axis=1)

Оставляю таблицу с признаками, которые нужны для обучения моделей

In [18]:
dataset.duplicated().sum()

49530

4 дубликата

In [19]:
dataset = dataset.drop_duplicates()
dataset.duplicated().sum()

0

Убрали дубликаты

In [20]:
dataset = dataset.query('RegistrationYear >= 1950')

Есть смысл удалить старые данные

In [21]:
dataset = dataset.query('RegistrationYear < 2023')

**Промежуточный вывод:**

Убрал аномальные значения, заполнил пропуски и убрал ненужные признаки 

## Обучение моделей

Перевод категориальных значений в количественные

In [22]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset.head(10)

,Price,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_yes
1,18300,2011,190,125000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2200,2004,109,150000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,14500,2014,125,30000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,999,1998,101,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
10,2000,2004,105,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11,2799,2005,140,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


Разделение на входные и целевой признаки

In [23]:
X = dataset.drop('Price', axis=1)
y = dataset['Price'] 

Разделение на тренировочную и тестовую выборки

In [24]:
features_train, features_test, target_train, target_test = train_test_split(X, y, test_size=0.2, random_state=42)

Линейная регрессия

In [25]:
%%time
start = time.time()
lr_model = LinearRegression()
CV = cross_val_score(lr_model, features_train, target_train, cv=5, scoring='neg_mean_squared_error')
#model_lr.fit(features_train, target_train)
end = time.time()
time_lr = round(end - start, 2)

CPU times: user 59.9 s, sys: 38.9 s, total: 1min 38s
Wall time: 2min


In [26]:
rmse_cv = round((-CV.mean()) ** 0.5, 2)

In [27]:
f"RMSE: {rmse_cv}"

'RMSE: 2978.34'

In [28]:
%%time
lr_model = LinearRegression()
lr_model.fit(features_train, target_train)

CPU times: user 13.7 s, sys: 8.45 s, total: 22.2 s
Wall time: 26.7 s


LinearRegression()

In [29]:
%%time
start = time.time()
predict_lr = lr_model.predict(features_test)
end = time.time()
time_lr_predict = round(end-start,2)

CPU times: user 127 ms, sys: 119 ms, total: 246 ms
Wall time: 393 ms


In [30]:
lr_rmse = mean_squared_error(target_test, predict_lr)**0.5
print('RMSE:', lr_rmse.round(2))

RMSE: 2999.29


LightGBM

Выделим признаки и целевой признак

In [31]:
X = dataset.drop('Price', axis=1)
y = dataset['Price']

Перевод категориальных признаков в тип category

In [32]:
for i in X.columns:
    col_type = X[i].dtype
    if col_type == 'object':
        X[i] = X[i].astype('category')

In [33]:
features_train, features_test, target_train, target_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
%%time
start = time.time()
model = lgbm.LGBMRegressor(random_state=42)
model.fit(features_train, target_train)
end = time.time()
time_lgbm = round(end-start,2)

CPU times: user 5min 11s, sys: 688 ms, total: 5min 11s
Wall time: 6min 28s


In [35]:
%%time
start = time.time()
predictions_train = model.predict(features_test)
end = time.time()
t_lgbm_predict = round(end - start, 2)

CPU times: user 595 ms, sys: 79.2 ms, total: 674 ms
Wall time: 901 ms


In [36]:
lgbm_rmse = mean_squared_error(target_test, predictions_train)**0.5
print('RMSE:', lgbm_rmse.round(2))

RMSE: 1780.46


Гиперпараметры для LightGBM

In [37]:
numerical_features = ['DateCreated', 'Price', 'RegistrationYear', 'Power', 'Kilometer', 'LastSeen']
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Repaired', 'Brand']

In [38]:
categorical_features = [col for col in list(dataset.columns) if col not in numerical_features]

In [39]:
%%time
train_data = lgbm.Dataset(features_train, label=target_train, 
                              free_raw_data=False,
                              categorical_feature=categorical_features)
test_data = lgbm.Dataset(features_test, label=target_test)

param_grid = {'learning_rate': [0.1, 0.3, 0.5, 0.7],
              'max_depth': [15, 20, 30, 35]}

lgbm = lgbm.LGBMRegressor(n_jobs = 8)

grid_search = GridSearchCV(estimator = lgbm, 
                           param_grid = param_grid, 
                           cv = 3,
                           n_jobs = -1, 
                           verbose = 0, 
                           scoring = 'neg_mean_squared_error')
grid_search.fit(features_train, target_train)

CPU times: user 48min 33s, sys: 29.2 s, total: 49min 2s
Wall time: 53min 46s


GridSearchCV(cv=3, estimator=LGBMRegressor(n_jobs=8), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5, 0.7],
                         'max_depth': [15, 20, 30, 35]},
             scoring='neg_mean_squared_error')

In [40]:
grid_search.best_params_

{'learning_rate': 0.3, 'max_depth': 20}

In [41]:
print('RMSE: {:.2f}'.format((-grid_search.best_score_) ** 0.5))

RMSE: 1710.81


In [42]:
%%time
preds_lgbm = grid_search.best_estimator_.predict(features_test)

CPU times: user 415 ms, sys: 69.3 ms, total: 484 ms
Wall time: 446 ms


In [43]:
lgbm = mean_squared_error(target_test, preds_lgbm)
lgbm_rmse = round((lgbm) ** 0.5, 2)
f"RMSE: {lgbm_rmse}"

'RMSE: 1727.01'

In [44]:
columns = ['RMSE', 'Скорость обучения, s', 'Скорость предсказания, s']

analysis_table = pd.DataFrame(index=['RMSE', 'Скорость обучения, s', 'Скорость предсказания, s'], columns=['LinearRegression','LightGBM'])
analysis_table['LinearRegression'] = rmse_cv, time_lr, time_lr_predict
analysis_table['LightGBM'] = lgbm_rmse, 280, t_lgbm_predict

analysis_table

,LinearRegression,LightGBM
RMSE,2978.34,1727.01
"Скорость обучения, s",120.39,280.00
"Скорость предсказания, s",0.39,0.90


**Общий вывод**

Что было сделано с данными: убрал аномальные значения, заполнил пропуски и убрал ненужные признаки

Были использованы две модели: LinearRegression и LightGBM

Скорость обучения: 74.2	и 280

Скорость предсказания: 0.12	0.64

RMSE: 2999.2 и 1727

Критерии, которые интересуют заказчика: 

- качество предсказания;
- скорость предсказания;
- время обучения.

Следовательно стоит использовать LightGBM, поскольку она имеет самые оптимальные характеристики.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей